In [ ]:
import cv2
import numpy as np

# Load the license plate image and template images of characters
plate_img = cv2.imread('ocr.jpg')
template_A = cv2.imread('template_A.png', 0)  # Grayscale template image of character 'A'

# Convert images to grayscale
plate_gray = cv2.cvtColor(plate_img, cv2.COLOR_BGR2GRAY)

# resize the image to have size ratio of 235:50, while keeping the original height
height, width = plate_gray.shape
plate_gray = cv2.resize(plate_gray, (int(width * 80 / height), 40))

# resize the template image to match the heigth of the license plate image
height, width = plate_gray.shape
height_template, width_template = template_A.shape

character_ratio = 0.9

template_A = cv2.resize(template_A, (int(width_template * height / height_template * character_ratio), int(height * character_ratio)))

# show both images
cv2.imshow('plate', plate_gray)
cv2.imshow('template', template_A)
cv2.waitKey(0)

In [ ]:
# Perform Non-Maximum Suppression
def apply_nms(boxes, threshold=0.5):
    if len(boxes) == 0:
        return []
    boxes = sorted(boxes, key=lambda x: x[2], reverse=True)
    picked_boxes = [boxes[0]]
    for box in boxes[1:]:
        add = True
        for picked_box in picked_boxes:
            x1 = max(box[0], picked_box[0])
            y1 = max(box[1], picked_box[1])
            x2 = min(box[0] + template_A.shape[1], picked_box[0] + template_A.shape[1])
            y2 = min(box[1] + template_A.shape[0], picked_box[1] + template_A.shape[0])
            intersection_area = max(0, x2 - x1) * max(0, y2 - y1)
            box_area = (template_A.shape[1]) * (template_A.shape[0])
            overlap_ratio = intersection_area / float(box_area)
            if overlap_ratio > threshold:
                add = False
                break
        if add:
            picked_boxes.append(box)
    return picked_boxes

In [ ]:
# Perform convolution for character 'A' as an example
match_A = cv2.matchTemplate(plate_gray, template_A, cv2.TM_CCOEFF_NORMED)

# Set a threshold to identify matches
threshold = 0.8
locations = np.where(match_A >= threshold)
# locations is a tuple of arrays, so we need to convert it to a n x 3 array format adding the third dimension as the value of the match
locations = list(zip(*locations[::-1]))
locations = [list(elem) + [match_A[elem[1], elem[0]]] for elem in locations]
locations

In [ ]:
# Apply NMS
picked_boxes = apply_nms(locations)

# Draw bounding boxes around detected positions after NMS
for loc in picked_boxes:
    cv2.rectangle(plate_gray, (loc[0], loc[1]), (loc[0] + template_A.shape[1], loc[1] + template_A.shape[0]), (0, 255, 0), 2)

# Display the result
cv2.imshow('Detected Characters', plate_gray)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
picked_boxes